# Gene Enrichment Analysis 

In [1]:
import pandas as pd 
import numpy as np
import os
import sklearn

In [156]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import matplotlib.patches as mpatches
from matplotlib.pyplot import gcf
import pickle
import statistics
import re 

Load pickle objects 

In [144]:
mod_d_human_mouse = pickle.load(open('mod_d_human_mouse.p','rb'))
data_space_jaccard = pickle.load(open('data_space_jaccard.p','rb'))
df_mapping = pickle.load(open('df_mapping.p','rb'))

### Load cluster of nanomaterials 
clusters_jaccard_40single = pickle.load(open('clusters_jaccard_40single.p','rb'))

### Load characterization of clusters 
grouping_cluster_jaccard40_single = pd.read_excel ('C:/Users/Gaia/Desktop/UNI/TIROCINIO/Grouping_cluster_jaccard40_single.xlsx')

## FunMappone

### Preparing input tables 

In [4]:
mapping_ensembl_genesymbol = pd.read_excel('C:/Users/Gaia/Desktop/UNI/TIROCINIO/Mapping EnsemblID - Genesymbol.xlsx')

#### Renaming the column names of data_space_jaccard with the one present in df_mapping in place. 

In [74]:
data_space_jaccard_mod = data_space_jaccard[data_space_jaccard.columns.drop(list(data_space_jaccard.filter(regex='P.Val', axis = 1)))]

for col in data_space_jaccard_mod.columns:
    col_split = col.split('-')[0]
    type_value = col.split('_')[-1]
  
    if col_split in df_mapping.index:
       
        new_name = df_mapping.loc[col_split,'Complete_info']
        data_space_jaccard_mod.rename(columns={col:new_name},inplace = True)

data_space_jaccard_mod = data_space_jaccard_mod.set_index('Genes')

C:\Users\Gaia\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [75]:
data_space_jaccard_mod 

,GSE112780_crocidolite_120ug_1mo,GSE112780_MWCNT_1ug_1mo_logFC,GSE112780_MWCNT_10ug_1mo,GSE112780_MWCNT_40ug_1mo,GSE112780_MWCNT_80ug_1mo,GSE112780_crocidolite_120ug_6mo,GSE112780_MWCNT_10ug_6mo,GSE112780_MWCNT_1ug_6mo_logFC,GSE112780_MWCNT_40ug_6mo,GSE112780_MWCNT_80ug_6mo,...,GSE92900_rCNT_10ug_24h,GSE92900_SES_10ug_24h,GSE92900_tCNT_10ug_24h,GSE92987_AuNPs_50uM_24h,GSE92987_AuNPs-HDM2_50uM_24h,GSE96720_GQD_50ug/ml_24h,GSE99929_Jurkat-GO_50ug/ml_24h,GSE99929_Jurkat-GONH2_50ug/ml_24h,GSE99929_THP1-GO_50ug/ml_24h,GSE99929_THP1-GONH2_50ug/ml_24h
Genes,,,,,,,,,,,,,,,,,,,,,
ENSG00000186141,0.129377,0.038586,0.060173,0.005947,-0.014359,0.061533,0.090022,0.051216,0.232877,0.060174,...,0.209523,0.054863,-0.088269,-1.014379,0.993773,1.229963,-0.000320,0.126782,0.018692,0.157015
ENSG00000134250,-0.089927,0.074649,0.028171,0.010845,-0.019129,-0.135967,-0.125878,-0.117532,-0.214893,-0.088255,...,-0.179172,0.004643,-0.093330,0.976848,-0.233548,1.810085,-0.318471,-0.110931,0.335438,-0.217906
ENSG00000143384,0.030705,0.012902,0.025958,-0.005395,-0.008615,-0.153426,-0.028909,-0.212354,-0.098006,-0.121506,...,-0.523542,-0.124049,-0.298915,-0.571782,-0.032959,-0.490165,-0.133376,0.028958,-0.103993,0.270971
ENSG00000131791,-0.199536,-0.022765,0.024714,-0.117255,0.010288,-0.278822,-0.183134,-0.002691,-0.129839,-0.225552,...,-0.016012,0.189290,0.112008,-0.846361,-0.319260,-1.000550,0.245724,-0.394620,-0.118810,-0.098901
ENSG00000023902,0.072309,-0.037498,-0.155610,-0.006213,0.125644,-0.338079,-0.188005,-0.160671,-0.212396,-0.098513,...,-0.036166,-0.148752,-0.061815,-0.607629,0.210257,0.706264,0.671616,0.181389,-0.329328,0.261824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000165219,-0.020427,0.047756,-0.057129,-0.076929,-0.078795,-0.012371,0.020806,-0.044153,0.061363,0.042882,...,-0.265919,0.094308,0.082977,-1.107976,-0.052194,-0.643626,0.022499,0.269165,0.136065,0.175482
ENSG00000106077,0.198431,-0.069461,-0.013199,0.104566,0.128885,0.061502,0.008707,-0.010948,-0.018248,0.027504,...,0.372326,1.376432,0.850080,0.473238,0.038895,-0.851018,0.122145,-0.130118,-0.017066,0.088867
ENSG00000160211,0.357998,-0.220650,0.218854,-0.287564,-0.195486,-0.042887,0.142017,0.318310,-0.161931,0.079120,...,0.412057,0.160696,0.023697,0.758641,0.052695,0.800444,0.151315,0.033919,0.029607,-0.303001


#### Define a function that given clusters it returns a dictionary with the union/intersection of all genes for each cluster and the average log fold change for each gene in each cluster. 

In [99]:
def gene_cluster_union(d,clusters,mapping_ensembl_genesymbol,data_space_jaccard_mod):
    c = {}
    l = {}
    for cl in clusters:
        l[cl] = []
        average_log = {}
        gene_set = set()
        for experiment in clusters[cl]:
           # print(experiment)
            for gene in d[experiment]:
              #  print(gene)
                if gene not in average_log.keys():
                    average_log[gene] = []
                
                average_log[gene].append(data_space_jaccard_mod.loc[gene,experiment])
                #print(average_log)
            gene_set = gene_set.union(d[experiment])
        
        for el in gene_set:
            mean = statistics.mean(average_log[el])
          #  print(average_log[el],mean,el)
            l[cl].append(mean)
                
        gene_set = list(gene_set)
        c[cl] = gene_set
    return c,l 

In [131]:
def gene_cluster_intersection(d,clusters,mapping_ensembl_genesymbol,data_space_jaccard_mod):
    
    l = {}
    inters = {}
    
    for cl in clusters:
        l[cl] = []
        gene_set = set()
        average_log = {}
        for i in range(len(clusters[cl])):
            #print(clusters[cl][i])
            for gene in d[clusters[cl][i]]:
                #print(gene)
                
                if gene not in average_log.keys():
                    average_log[gene] = []
                
                average_log[gene].append(data_space_jaccard_mod.loc[gene,clusters[cl][i]])
            
            if i == 0:
                gene_set = gene_set.union(d[clusters[cl][i]])
            else:
                gene_set = gene_set.intersection(d[clusters[cl][i]])
        gene_set = list(gene_set)
        
        for el in gene_set:
            mean = statistics.mean(average_log[el])
          #  print(average_log[el],mean,el)
            l[cl].append(mean)
        
        inters[cl] = gene_set
        
    return inters,l

In [103]:
gene_cluster_union_jaccard40single,fold_change_jaccard40_single_union = gene_cluster_union(mod_d_human_mouse,clusters_jaccard_40single,mapping_ensembl_genesymbol,data_space_jaccard_mod)

In [133]:
gene_cluster_intersection_jaccard40single, fold_change_jaccard40_single_inter = gene_cluster_intersection(mod_d_human_mouse,clusters_jaccard_40single,mapping_ensembl_genesymbol,data_space_jaccard_mod)

In [134]:
gene_cluster_intersection_jaccard40single

{0: ['ENSG00000102265'],
 1: ['ENSG00000138182',
  'ENSG00000145386',
  'ENSG00000136982',
  'ENSG00000168765',
  'ENSG00000073111',
  'ENSG00000111602',
  'ENSG00000113810',
  'ENSG00000013810',
  'ENSG00000135069',
  'ENSG00000100297',
  'ENSG00000175130',
  'ENSG00000101868',
  'ENSG00000117399',
  'ENSG00000141644',
  'ENSG00000166851',
  'ENSG00000096060',
  'ENSG00000133027',
  'ENSG00000136824',
  'ENSG00000174371',
  'ENSG00000062716',
  'ENSG00000117724',
  'ENSG00000124225',
  'ENSG00000090889',
  'ENSG00000099622',
  'ENSG00000105173',
  'ENSG00000168067',
  'ENSG00000167900',
  'ENSG00000168078',
  'ENSG00000170312',
  'ENSG00000165795',
  'ENSG00000184445',
  'ENSG00000100526',
  'ENSG00000088325',
  'ENSG00000112715',
  'ENSG00000100097',
  'ENSG00000090006',
  'ENSG00000128951',
  'ENSG00000163947',
  'ENSG00000127603',
  'ENSG00000174442',
  'ENSG00000177595',
  'ENSG00000125319',
  'ENSG00000185222',
  'ENSG00000143819',
  'ENSG00000103966',
  'ENSG00000165304',
  'ENS

In [135]:
fold_change_jaccard40_single_inter

{0: [1.2629941913192826],
 1: [1.1988854326817109,
  2.2131922399313133,
  0.9306847987087837,
  -1.0076707087262702,
  1.1807621896631308,
  1.1229416673647565,
  0.9670501054557026,
  1.6486754787646714,
  1.2464257041958018,
  1.3020853127439516,
  0.9387440512472573,
  0.8786468973366959,
  1.9592107094410394,
  0.92701815668612,
  1.9483743867410808,
  1.2710248404123394,
  1.0311448196084465,
  1.1485039328699715,
  1.6849936072754221,
  1.0973945362363589,
  1.73608320584552,
  1.0022170072167491,
  1.1363339152364835,
  -1.418442367568615,
  1.4456180635028477,
  -0.7549977802256923,
  1.803762999853783,
  1.9425035766369456,
  1.8375631425237955,
  -0.9195077559485649,
  1.4491565304783873,
  1.4380484300779812,
  1.855874068406066,
  -0.9395583323772863,
  1.2474895227312137,
  -1.2330486781005263,
  0.9756365221264246,
  -0.7810720397451343,
  -0.9031481104608972,
  1.2897207996266296,
  0.9021428631022889,
  1.5344077951722848,
  0.9284575591854867,
  -1.0997155964997762,
 

#### Define a function that given the gene clusters it returns the input table for FunMappone tool, the last sheet is the grouping of clusters based on 'only in vivo, only in ex vivo, only in vitro, mixed'.  

In [196]:
def grouping_funmappone(grouping_characteristic,grouping_cluster):
    cluster = {}
    grouping = {}
    cluster['Cluster'] = []
    grouping['Grouping'] = []
    for cl in grouping_cluster.columns:
      #  print(cl)
        cluster['Cluster'].append(cl)
        s = grouping_cluster.loc[grouping_characteristic,cl]
        if re.search('.*100',s):
            if re.search('in vivo',s): 
                grouping['Grouping'].append('in vivo')
            elif re.search('ex vivo',s):
                grouping['Grouping'].append('ex vivo')
            elif re.search('in vitro',s):
                grouping['Grouping'].append('in vitro')
        else:
            grouping['Grouping'].append('mixed')
    
    df = pd.DataFrame(cluster)
    df['Grouping'] = grouping['Grouping']
    return df 

In [197]:
def funmappone_input(gene_cluster,name_file,foldchange_cluster,grouping_cluster):
    writer = pd.ExcelWriter(name_file, engine='xlsxwriter')
    
    for key in gene_cluster:
     
        df = pd.DataFrame(gene_cluster[key])
        df = df.rename(columns={0:'Symbol'})
        df['T-stats'] = foldchange_cluster[key]
 
        df.to_excel(writer, index = False, sheet_name=str(key))
    
    grouping_sheet = grouping_funmappone(grouping_cluster.index[1],grouping_cluster)
    grouping_sheet.to_excel(writer, index = False, sheet_name = 'Grouping')
    writer.save()
    

In [198]:
funmappone_input(gene_cluster_union_jaccard40single,'C:/Users/Gaia/Desktop/UNI/TIROCINIO/funmappone_union_jaccard40_single.xlsx',fold_change_jaccard40_single_union,grouping_cluster_jaccard40_single)
funmappone_input(gene_cluster_intersection_jaccard40single,'C:/Users/Gaia/Desktop/UNI/TIROCINIO/funmappone_intersection_jaccard40_single.xlsx',fold_change_jaccard40_single_inter,grouping_cluster_jaccard40_single)


## GSEA Input 
with http://www.webgestalt.org/#

In [199]:
gene_cluster_union_jaccard40single

{0: ['ENSG00000148926',
  'ENSG00000162772',
  'ENSG00000102265',
  'ENSG00000134107'],
 1: ['ENSG00000171453',
  'ENSG00000145293',
  'ENSG00000170854',
  'ENSG00000005189',
  'ENSG00000105698',
  'ENSG00000105926',
  'ENSG00000143013',
  'ENSG00000106367',
  'ENSG00000166881',
  'ENSG00000114850',
  'ENSG00000135720',
  'ENSG00000109184',
  'ENSG00000137880',
  'ENSG00000129933',
  'ENSG00000169826',
  'ENSG00000123352',
  'ENSG00000166024',
  'ENSG00000117748',
  'ENSG00000124207',
  'ENSG00000145687',
  'ENSG00000128791',
  'ENSG00000185650',
  'ENSG00000133789',
  'ENSG00000165795',
  'ENSG00000185022',
  'ENSG00000142208',
  'ENSG00000150403',
  'ENSG00000051108',
  'ENSG00000164068',
  'ENSG00000188313',
  'ENSG00000147872',
  'ENSG00000114503',
  'ENSG00000120438',
  'ENSG00000100325',
  'ENSG00000106460',
  'ENSG00000151465',
  'ENSG00000167699',
  'ENSG00000133265',
  'ENSG00000101400',
  'ENSG00000167325',
  'ENSG00000136270',
  'ENSG00000005339',
  'ENSG00000196372',
  'ENS

Define a function that for each cluster it returns a ranked list of genes based on the average log fold change. 

In [217]:
def GSEA_input(gene_cluster, logfold_change,file_path):
    
    for cl in gene_cluster:
        print(cl)
        df = pd.DataFrame(gene_cluster[cl])
        df = df.rename(columns={0:'gene IDs'})
        df['score'] = logfold_change[cl] 
        
        df = df.sort_values('score')
        print(df)
    
        df.to_csv(file_path + str(cl) + '.rnk', index=None, sep='\t')

In [218]:
GSEA_input(gene_cluster_union_jaccard40single,fold_change_jaccard40_single_union,'C:/Users/Gaia/Desktop/UNI/TIROCINIO/GSEA input/')

0
          gene IDs     score
0  ENSG00000148926 -0.790300
3  ENSG00000134107  0.718981
1  ENSG00000162772  0.848893
2  ENSG00000102265  1.262994
1
             gene IDs     score
711   ENSG00000105516 -1.954589
677   ENSG00000132640 -1.501882
737   ENSG00000099622 -1.418442
196   ENSG00000167074 -1.339058
678   ENSG00000169231 -1.250877
...               ...       ...
617   ENSG00000134690  2.206889
1016  ENSG00000145386  2.213192
96    ENSG00000132294  2.218203
263   ENSG00000089685  2.362569
428   ENSG00000102265  3.553485

[1234 rows x 2 columns]
2
            gene IDs     score
157  ENSG00000106683 -1.482894
260  ENSG00000103657 -1.419622
154  ENSG00000136811 -1.350555
81   ENSG00000105516 -1.328069
162  ENSG00000198799 -1.256557
..               ...       ...
101  ENSG00000167900  1.584895
234  ENSG00000089685  1.617568
93   ENSG00000145386  1.635522
12   ENSG00000134690  1.658198
104  ENSG00000102265  2.756634

[262 rows x 2 columns]
3
            gene IDs     score
964  ENSG00

26
            gene IDs     score
140  ENSG00000111276 -1.614837
31   ENSG00000179041 -1.075298
216  ENSG00000137504 -0.959360
136  ENSG00000151093 -0.853359
114  ENSG00000164587 -0.829590
..               ...       ...
111  ENSG00000171223  2.630070
204  ENSG00000116741  2.637309
9    ENSG00000175197  2.942017
45   ENSG00000132002  2.952951
124  ENSG00000120738  3.259653

[232 rows x 2 columns]
27
            gene IDs     score
68   ENSG00000112312 -1.014039
67   ENSG00000110344 -0.861899
19   ENSG00000101224 -0.835396
36   ENSG00000169136 -0.809905
26   ENSG00000189060 -0.799643
..               ...       ...
119  ENSG00000151929  2.066883
10   ENSG00000198431  2.148067
40   ENSG00000132002  2.785277
9    ENSG00000120129  2.790694
82   ENSG00000162772  2.885521

[136 rows x 2 columns]
28
            gene IDs     score
21   ENSG00000189060 -1.205268
226  ENSG00000108389 -1.067840
179  ENSG00000087586 -0.919167
10   ENSG00000080546 -0.878421
12   ENSG00000177239 -0.859486
..           